In [48]:
import torch
import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline

In [ ]:
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [50]:
# Physics
b_max = 0.3  # maximum ice balance m/yr
grad_b = 0.001  # gradient of ice balance as a function of elevation. yr^-1

z_ELA = 1200  # equilibrium line altitude
Lx = 1e5  # model length, m
ttot = 3e3  # total time
rhog = 910 * 9.81  # ice density kg/m^3 * gravity m/s^{-2}
fd = 1e-15  # physical constant for ice diffusivity, Pa^3/y
kD = fd * rhog**3  # lump all constants for diffusivity together

# Numerics
nx = 201  # number of cells
dx = Lx / (nx - 1)  # number of cells 
dtmax = 1      # initial dt, will be changed within loop, yr
dt    = dtmax  # initial dt, will be changed within loop, yr
x = torch.linspace(0, Lx, nx, device=device)  # x-coordinates
nout = 500  # frequency of plotting

# Initialization
Z_bed = -1000 * torch.log(x + 1000) + 11522.8  # topography
H = torch.zeros(nx, device=device)  # ice thickness
time = 0.0  # initialize time
nplot = 0  # counter for plotting

Z = Z_bed + H  # ice surface

time = 0
it = 0
# Epsilon to avoid division by zero
epsilon = 1e-12

In [51]:
# Visualization function using tolist()
def visualize_glacier(x, Z, Z_bed, z_ELA, b, time):
    Z_list = Z.cpu().tolist()  # Convert to list
    Z_bed_list = Z_bed.cpu().tolist()  # Convert to list
    x_list = x.cpu().tolist()  # Convert to list
    b_list = b.cpu().tolist()  # Convert to list
    clear_output(wait=True)  # Clear the previous output in the notebook
    
    plt.figure(2, figsize=(7, 5), dpi=200)
    plt.subplot(2, 2, 1)
    plt.plot([xi / 1000 for xi in x_list], Z_list, 'b', linewidth=2) 
    plt.plot([xi / 1000 for xi in x_list], Z_bed_list, 'k', linewidth=1)
    plt.plot([x_list[0] / 1000, x_list[-1] / 1000], [z_ELA, z_ELA], 'g')
    plt.ylim([min(Z_bed_list), max(Z_bed_list)])
    plt.ylabel('Elevation, m')
    plt.title('Glacier après ' + str(int(time)) + ' années')

    plt.subplot(2, 2, 2)
    plt.plot([min(grad_b * (zi - z_ELA), b_max) for zi in Z_list], Z_list, '--k', linewidth=2)
    plt.plot([min(b_list), b_max + 0.3], [z_ELA, z_ELA], 'g')
    plt.plot([0, 0], [min(Z_bed_list), max(Z_bed_list)], 'k', linewidth=1)
    plt.ylim([min(Z_bed_list), max(Z_bed_list)])
    plt.title('Fonction bilan de masse') 
    plt.xlabel('Bilan de masse, m/a')  

    plt.subplot(2, 2, 3, aspect=20.0)
    plt.plot([xi / 1000 for xi in x_list], b_list, 'b', linewidth=2)
    plt.plot([x_list[0] / 1000, x_list[-1] / 1000], [0, 0], 'k', linewidth=1)
    plt.title('Bilan de masse effectif')
    plt.xlabel('Distance, km') 
    plt.ylabel('Bilan de masse, m/a') 

    plt.show()

In [ ]:
# Time loop
while time < ttot:

    # compute dHdt due to diffusion
    H_av = 0.5 * (H[:-1] + H[1:])  # average height between cells (nx-1)
    D = kD * H_av**5 * (torch.diff(Z) / dx)**2  # diffusivity (nx-1)
    qx = -D * torch.diff(Z) / dx  # ice flux
    dHdt = -torch.diff(qx) / dx  # change in ice thickness from flow

    # update time step as function of D
    max_D = torch.max(D).item()
    dt = min(dtmax, dx**2 / (2.1 * (max_D + epsilon)))  # update time step (added epsilon to avoid division by zero)

    # update thickness of ice (iceflow)
    H[1:-1] += dt * dHdt  # update ice thickness from flow

    # update ice thickness (mass balance)
    b = torch.minimum(grad_b * (Z - z_ELA), torch.tensor(b_max, device=device)) 
    H[1:-1] += dt * b[1:-1] 

    H[H < 0] = 0  # set any negative thickness to 0

    # Boundary conditions
    H[0] = 0
    H[-1] = 0

    Z = Z_bed + H  # update ice surface
    time += dt  # update time

    it += 1

    # Call visualization function
    if it % nout == 0:

        visualize_glacier(x, Z, Z_bed, z_ELA, b, time)

In [35]:
# Sav e the observations.
# torch.save(H,'observed_thk.pt')

In [45]:
# Define a small epsilon to avoid division by zero or log of zero
epsilon = 1e-8

# Function to perform forward simulation of glacier thickness with differentiable operations
def forward_simulation(b_max, grad_b, z_ELA, nx=201, Lx=1e5, ttot=3e3, dtmax=1, device='cpu'):
    # Physics constants
    rhog = 910 * 9.81  # ice density kg/m^3 * gravity m/s^{-2}
    fd = 1e-15  # physical constant for ice diffusivity, Pa^3/y
    kD = fd * rhog**3  # lump all constants for diffusivity together
    dx = Lx / (nx - 1)  # cell size
    
    # Initialization
    x = torch.linspace(0, Lx, nx, device=device)  # x-coordinates
    Z_bed = -1000 * torch.log(x + 1000 + epsilon) + 11522.8  # topography (avoid log(0))
    H = torch.zeros(nx, device=device)  # initial ice thickness
    Z = Z_bed + H  # ice surface
    
    time = 0.0
    while time < ttot:
        # Compute dHdt due to diffusion
        H_av = 0.5 * (H[:-1] + H[1:]) + epsilon  # avoid zero in averaging height
        qx = - torch.diff(Z) / (dx + epsilon)  # ice flux
        dHdt = -torch.diff(qx) / (dx + epsilon)  # change in ice thickness from flow
        
        # Update time step as function of D
        max_D = 0.1
        dt = min(dtmax, dx**2 / (2.1 * max_D))  # avoid division by zero
        
        # Use out-of-place operations to update ice thickness from flow and mass balance
        H_new = H.clone()  # create a new tensor to avoid in-place operations
        H_new[1:-1] += dt * dHdt  # ice thickness from flow
        
        # Handle minimum ice balance using a smooth approximation for differentiability
        b = torch.tanh(grad_b * (Z - z_ELA)) * b_max  # Smooth approximation for mass balance
        H_new[1:-1] += dt * b[1:-1]  # ice thickness from mass balance
        
        H_new[H_new < 0] = 0  # ensure ice thickness is non-negative
        H_new[0] = H_new[-1] = 0  # boundary conditions

        # Update ice surface
        Z = Z_bed + H_new
        H = H_new  # Assign updated thickness for the next iteration
        time += dt  # update time
        # if it % nout == 0:
        #     visualize_glacier(x, Z, Z_bed, z_ELA, b, time)
    
    return H

In [ ]:
# test the forward run 
forward_simulation(b_max,grad_b,z_ELA)

In [ ]:
# Initial guesses for inversion problem
b_max = 0.3
grad_b = 0.001
z_ELA = torch.tensor(1000.0, requires_grad=True, device='cpu')

# Observed glacier thickness (assumed already loaded as observed_thk tensor)
observed_thk = torch.load('observed_thk.pt', weights_only=True).to('cpu') # Ensure it's on the right device

# Optimizer setup
optimizer = torch.optim.Adam([z_ELA], lr=1)

# Regularization weight (tune this value)
reg_lambda = 0.1

# Optimization loop
n_iterations = 50



# Initialize lists to track loss components
total_loss_history = []
data_fidelity_history = []
regularization_history = []

for i in range(n_iterations):
    optimizer.zero_grad()  # Zero gradients
    
    # Perform forward simulation
    H_simulated = forward_simulation(b_max, grad_b, z_ELA)
    
    # Compute data fidelity and regularization
    data_fidelity = torch.mean((H_simulated - observed_thk) ** 2)
    smoothness_reg = torch.sum((H_simulated[1:] - H_simulated[:-1]) ** 2)
    
    # Compute total loss
    loss = data_fidelity + reg_lambda * smoothness_reg
    
    # Backpropagate loss and update parameters
    loss.backward()

    optimizer.step()
    
    # Store loss components for plotting later
    total_loss_history.append(loss.item())
    data_fidelity_history.append(data_fidelity.item())
    regularization_history.append((reg_lambda * smoothness_reg).item())
    
    # Print loss and current parameters every 50 iterations
    if (i + 1) % 5 == 0:
        print(f"Iteration {i+1}/{n_iterations}, Loss: {loss.item()}")
        print(f"z_ELA: {z_ELA.item()}")


In [31]:
# Function to plot the loss function components
def plot_loss_components(total_loss_history, data_fidelity_history, regularization_history):
    plt.figure(figsize=(10, 6))

    # Plot the total loss, data fidelity, and regularization term
    plt.plot(total_loss_history, label='Total Loss', color='b', linewidth=2)
    plt.plot(data_fidelity_history, label='Data Fidelity', color='g', linestyle='--', linewidth=2)
    plt.plot(regularization_history, label='Regularization (Smoothness)', color='r', linestyle='-.', linewidth=2)

    # Add labels and legend
    plt.xlabel('Iteration')
    plt.ylabel('Loss Value')
    plt.title('Loss Function Components Over Iterations')
    plt.legend()

    # Show the plot
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# After the optimization loop
plot_loss_components(total_loss_history, data_fidelity_history, regularization_history)
